In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_path = "/home/uuz5szh/Desktop/test/code/codeGenerator/data/Qwen"

tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    trust_remote_code=True,
    torch_dtype=torch.float32
).to("cpu")

model.eval()


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [2]:
from docx import Document
from PyPDF2 import PdfReader  # 新增导入

def read_input_file(file_path):
    if file_path.endswith(".txt"):
        with open(file_path, "r", encoding="utf-8") as f:
            return f.read()
    elif file_path.endswith(".docx"):
        doc = Document(file_path)
        return "\n".join([p.text for p in doc.paragraphs])
    elif file_path.endswith(".pdf"):
        reader = PdfReader(file_path)
        text = ""
        for page in reader.pages:
            text += page.extract_text() + "\n"
        return text.strip()
    else:
        raise ValueError("Unsupported file type: please use .txt, .docx or .pdf")


In [6]:
def generate_technical_doc(user_need_text):
    prompt = f"""\
You are an AI technical writer. Based on the following user request, write a detailed software or technical documentation in markdown format.

Each part should include:
1. **Function description**
2. **Technical details**
3. **Input and Output**
4. **Original requirement excerpt**

# User Request:
{user_need_text.strip()}

# Documentation:
"""

    inputs = tokenizer(prompt, return_tensors="pt")
    output = model.generate(
        **inputs,
        max_new_tokens=1024,
        do_sample=True,
        top_p=0.9,
        temperature=0.7,
        eos_token_id=tokenizer.eos_token_id
    )

    result = tokenizer.decode(output[0], skip_special_tokens=True)
    return result.split("Technical File：")[-1].strip()


In [4]:
def save_output(text, output_path):
    with open(output_path, "w", encoding="utf-8") as f:
        f.write(text)
    print(f"Output has been saved to: {output_path}")


In [7]:

input_path = "input.txt"  # 或 "sample.txt"
output_path = "整理后的技术文档.txt"

# 运行
user_need = read_input_file(input_path)
tech_doc = generate_technical_doc(user_need)
print("==== 整理结果预览 ====\n")
print(tech_doc[:1000])  # 只预览前1000字符

# 保存输出
save_output(tech_doc, output_path)


Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


==== 整理结果预览 ====

You are an AI technical writer. Based on the following user request, write a detailed software or technical documentation in markdown format.

Each part should include:
1. **Function description**
2. **Technical details**
3. **Input and Output**
4. **Original requirement excerpt**

# User Request:
5.1.3 快速启动模式
ICU 需要支持快速启动 STR 模式
1)
2)
3)
4)
5)
此模式下，MCU 休眠，SOC 处于 STR 模式，DRAM 保持上电；
可通过 IGN ON 和 CAN 网络信号将主机从 STR 模式唤醒至 TempRun/RUN 模式
STR 模式下的主机静态电流<8mA
主机从 STR 模式恢复到 TempRun/RUN 模式的时间需小于 5S
STR 持续时间可配置标定,默认持续时间为 72 小时，若持续时间为 0，则禁用 STR，若
持续时间设置为 255，则表示永久保持 STR 不关闭,支持智能补电的车型 STR 永不关闭。
12CVTC-IVI-Func-013: 电源管理
6)
Version 0.2.4
当主机在 STR 模式下的持续时间超时后，若车辆仍处于熄火状态，则主机进入 Seep 模式
5.1.4 快速关机模式
NA
5.2 主机背光联动
主机屏背光在电压状态变高时，需要联动调低背光；在电压状态变低时，需要联动调高背
光。这个功能由屏幕自己的保护措施来实现。
5.3 开机动画

IVI 系统冷启动时间：6125 平台<25S
6125plus 平台<16s（开机后到显示 Launcher 界面）
。
首次接通 Battery，ACC ON 时，系统启动后执行开机动画，动画结束后进入工作模式，进
入 HOME 界面。

开机动画显示时机：
系统仅在每次从 Standby 模式到工作模式或 TempRun 模式的过程中，播放开机动画。如
果没有进入过睡眠模式，再次从“待机模式”到其他工作